# KDNuggets Blockchain in 7 Steps Blogpost

https://www.kdnuggets.com/2018/04/blockchain-explained-7-python-functions.html

> At the heart of the blockchain is the hashing function

In [1]:
def hash_function(k):
    """Hashes our transaction"""
    if type(k) is not str:
        k = json.dumps(k, sort_keys=True)
        
    return hashlib.sha256(k).hexdigest()

In [ ]:
def update_state(transaction, state):
    state = state.copy()
    
    for key in transaction:
        if key in state.keys():
            state[key] += 